In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-43]) 
sys.path.append(cpath[0][:-43]+'/algorithms')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular

abalone = Dataset()

In [3]:
X, y, feature_names, class_names = abalone.load_abalone(type='classification',ys='reduced')

In [4]:
y = [int(i) for i in y]
y = np.array(y)
#one_h_percent = int(min(10*len(X)/100,100))
#print("Instances:",one_h_percent)

In [8]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf' : [10],
        'n_estimators': [10]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            return rule, cover, def_predictions, time.time() - ts

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = cpath[0][:-43]+'/algorithms/CHIRPS'
        temp_y = np.array([int(i+1) for i in y_train])
        temp_frame = pd.DataFrame(np.hstack((train,temp_y.reshape(len(temp_y),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'abalon', random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time() - chts)/len(test)
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance, reduction=True, ar_algorithm='fpgrowth', cl_algorithm='kmedoids', method='123')[-1]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1

    gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    
    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)  
    
    ktime = time.time()
    for test_ind in range(len(test)):
        if test_ind % 100 ==0:
            print(round(test_ind/(len(test))*100,2),'in:', time.time()-ktime)
            ktime = time.time()
            
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(test_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te = method(np.concatenate((x_test_temp[test_ind:test_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response

In [9]:
def print_current(total_results):
    full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    for i in total_results:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/len(i[4][name]))
            rule_length[name].append(i[2][name]/len(i[4][name]))
            if len(i[3][name]) >= 1:
                f_precision[name].append(np.array(i[3][name]))                
            f_time[name].append(np.array(i[4][name]))
    for name, method in i[0].items():
        print(name,np.array(full_coverage[name]).mean(),',',
              np.array(rule_length[name]).mean(),',',
              np.array(f_precision[name]).mean(),',',
              np.array(f_time[name]).mean())

In [10]:
total_results = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)
results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
total_results.append(results)
total_results.append(results)
print_current(total_results)

len self.path_detail
806
Working on CHIRPS for instance 0 of 806
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
reduced 1273 patterns out of 6430 by numeric redundancy
found 5157 patterns from 10 trees for batch_idx 0
start score sort for batch_idx 0 (5157) patterns
start merge rule for batch_idx 0 (5157) patterns
[('Shell', True, 0.18699), ('Whole', False, 0.04328)]
0.9840579710144928 0.2111158309925273 0.0022100922464078163 0.007903936768505052
merge complete for batch_idx 0 (5157) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
reduced 1500 patterns out of 5434 by numeric redundancy
found 3934 patterns from 7 trees for batch_idx 1
start score sort for batch_idx 1 (3934) patterns
start merge rule for batch_idx 1 (3934) patterns
[('Shell', True, 0.4191)]
0.8363988383349468 0.41954515650871643 0.13745596711832953 0.1545487106017258
merge complete for batch_idx 1 (3934) patterns
start get e

start merge rule for batch_idx 16 (6429) patterns
[('Shell', True, 0.25047)]
0.8861209964412812 0.3028888225554203 0.013649268877459585 0.02649678655992476
merge complete for batch_idx 16 (6429) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
reduced 370 patterns out of 1690 by numeric redundancy
found 1320 patterns from 8 trees for batch_idx 17
start score sort for batch_idx 17 (1320) patterns
start merge rule for batch_idx 17 (1320) patterns
[('Shucked', True, 0.4083), ('Shell', False, 0.39019), ('Whole', False, 0.4342)]
0.8297872340425532 0.027758601595757344 0.05926257895375943 0.07403314917127071
merge complete for batch_idx 17 (1320) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
reduced 1005 patterns out of 3892 by numeric redundancy
found 2887 patterns from 9 trees for batch_idx 18
start score sort for batch_idx 18 (2887) patterns


start merge rule for batch_idx 32 (2212) patterns
[('Shell', True, 0.53572)]
0.8015903067020068 0.38011756331657137 0.06434209603249186 0.10410517842769723
merge complete for batch_idx 32 (2212) patterns
start get explainer for batch_idx 32
as_chirps for batch_idx 33
start mining for batch_idx 33 with support = 0.1
reduced 1033 patterns out of 3803 by numeric redundancy
found 2770 patterns from 7 trees for batch_idx 33
start score sort for batch_idx 33 (2770) patterns
start merge rule for batch_idx 33 (2770) patterns
[('Shell', True, 0.4891)]
0.8155102040816327 0.4087493845019505 0.04678526861724241 0.0614352090862129
merge complete for batch_idx 33 (2770) patterns
start get explainer for batch_idx 33
as_chirps for batch_idx 34
start mining for batch_idx 34 with support = 0.1
reduced 42 patterns out of 263 by numeric redundancy
found 221 patterns from 7 trees for batch_idx 34
start score sort for batch_idx 34 (221) patterns
start merge rule for batch_idx 34 (221) patterns
[('Shell', Tr

[('Shell', False, 0.58038), ('Shucked', True, 0.5243), ('Whole', False, 0.56621), ('Height', False, 0.6622340232133865)]
0.46511627906976744 0.012017224386863012 0.6218786914713444 0.7073170731707328
merge complete for batch_idx 49 (99) patterns
start get explainer for batch_idx 49
Working on CHIRPS for instance 50 of 806
as_chirps for batch_idx 50
start mining for batch_idx 50 with support = 0.1
reduced 1560 patterns out of 3832 by numeric redundancy
found 2272 patterns from 8 trees for batch_idx 50
start score sort for batch_idx 50 (2272) patterns
start merge rule for batch_idx 50 (2272) patterns
[('Shell', True, 0.53099), ('Shucked', False, 0.45411)]
0.9193899782135077 0.13625309024668394 0.17876892846083753 0.19409704852426213
merge complete for batch_idx 50 (2272) patterns
start get explainer for batch_idx 50
as_chirps for batch_idx 51
start mining for batch_idx 51 with support = 0.1
reduced 139 patterns out of 1522 by numeric redundancy
found 1383 patterns from 4 trees for batch_

[('Shucked', True, 0.43829), ('Whole', False, 0.45425)]
0.8727272727272727 0.03272089150443962 0.02570871076475449 0.03897032534858578
merge complete for batch_idx 65 (2214) patterns
start get explainer for batch_idx 65
as_chirps for batch_idx 66
start mining for batch_idx 66 with support = 0.1
reduced 127 patterns out of 1372 by numeric redundancy
found 1245 patterns from 6 trees for batch_idx 66
start score sort for batch_idx 66 (1245) patterns
start merge rule for batch_idx 66 (1245) patterns
[('Shucked', True, 0.29146), ('Shell', False, 0.37205)]
0.8918918918918919 0.02166247320653248 0.07972406201317161 0.07659269863994447
merge complete for batch_idx 66 (1245) patterns
start get explainer for batch_idx 66
as_chirps for batch_idx 67
start mining for batch_idx 67 with support = 0.1
reduced 800 patterns out of 7823 by numeric redundancy
found 7023 patterns from 10 trees for batch_idx 67
start score sort for batch_idx 67 (7023) patterns
start merge rule for batch_idx 67 (7023) patter

...

start merge rule for batch_idx 741 (2621) patterns
[('Shell', True, 0.22752)]
0.8775913129318855 0.27415750116721394 0.013645183652410667 0.035785857429144
merge complete for batch_idx 741 (2621) patterns
start get explainer for batch_idx 741
as_chirps for batch_idx 742
start mining for batch_idx 742 with support = 0.1
reduced 16 patterns out of 623 by numeric redundancy
found 607 patterns from 3 trees for batch_idx 742
start score sort for batch_idx 742 (607) patterns
start merge rule for batch_idx 742 (607) patterns
[('Shucked', True, 0.36542), ('Whole', False, 0.41423)]
0.7592592592592593 0.01544085151218139 0.024691134826566777 0.02668759811616935
merge complete for batch_idx 742 (607) patterns
start get explainer for batch_idx 742
as_chirps for batch_idx 743
start mining for batch_idx 743 with support = 0.1
reduced 1773 patterns out of 4462 by numeric redundancy
found 2689 patterns from 10 trees for batch_idx 743
start score sort for batch_idx 743 (2689) patterns
start merge rule 

start merge rule for batch_idx 757 (4329) patterns
[('Shell', True, 0.28051)]
0.8807896735003796 0.34293988656976876 0.10488415596239935 0.13235032865516663
merge complete for batch_idx 757 (4329) patterns
start get explainer for batch_idx 757
as_chirps for batch_idx 758
start mining for batch_idx 758 with support = 0.1
reduced 1315 patterns out of 5329 by numeric redundancy
found 4014 patterns from 9 trees for batch_idx 758
start score sort for batch_idx 758 (4014) patterns
start merge rule for batch_idx 758 (4014) patterns
[('Shell', True, 0.61202), ('Whole', True, 0.69748), ('Length', False, 0.78571), ('Shucked', False, 0.56933), ('Shucked', False, 0.49926), ('Viscera', False, 0.52451), ('Viscera', True, 0.64888)]
0.88 0.021895466107424502 0.052088303788515346 0.08078961798575121
merge complete for batch_idx 758 (4014) patterns
start get explainer for batch_idx 758
as_chirps for batch_idx 759
start mining for batch_idx 759 with support = 0.1
reduced 347 patterns out of 3241 by numer

start merge rule for batch_idx 773 (1201) patterns
[('Shell', False, 0.41244), ('Shucked', True, 0.43555), ('Whole', False, 0.45031)]
0.8828828828828829 0.033043422236068175 0.09925699406893324 0.12056737588652763
merge complete for batch_idx 773 (1201) patterns
start get explainer for batch_idx 773
as_chirps for batch_idx 774
start mining for batch_idx 774 with support = 0.1
reduced 1181 patterns out of 5553 by numeric redundancy
found 4372 patterns from 10 trees for batch_idx 774
start score sort for batch_idx 774 (4372) patterns
start merge rule for batch_idx 774 (4372) patterns
[('Shell', True, 0.23549)]
0.8804554079696395 0.2846279840493521 0.04584812131586984 0.07367877431570959
merge complete for batch_idx 774 (4372) patterns
start get explainer for batch_idx 774
Working on CHIRPS for instance 775 of 806
as_chirps for batch_idx 775
start mining for batch_idx 775 with support = 0.1
reduced 1103 patterns out of 3055 by numeric redundancy
found 1952 patterns from 10 trees for batch

[('Shell', True, 0.02327)]
0.9393939393939394 0.019218846869187848 0.033279936439029524 0.12195121951219526
merge complete for batch_idx 789 (84) patterns
start get explainer for batch_idx 789
Working on CHIRPS for instance 790 of 806
as_chirps for batch_idx 790
start mining for batch_idx 790 with support = 0.1
reduced 1467 patterns out of 5350 by numeric redundancy
found 3883 patterns from 10 trees for batch_idx 790
start score sort for batch_idx 790 (3883) patterns
start merge rule for batch_idx 790 (3883) patterns
[('Shell', True, 0.16236), ('Shell', False, 0.10192)]
0.9874213836477987 0.09733415995040298 0.017424623885647923 0.04350170771166177
merge complete for batch_idx 790 (3883) patterns
start get explainer for batch_idx 790
as_chirps for batch_idx 791
start mining for batch_idx 791 with support = 0.1
reduced 1449 patterns out of 5499 by numeric redundancy
found 4050 patterns from 9 trees for batch_idx 791
start score sort for batch_idx 791 (4050) patterns
start merge rule for

start merge rule for batch_idx 805 (903) patterns
[('Shell', True, 0.39385), ('Shell', False, 0.29519), ('Shucked', True, 0.22998)]
0.9130434782608695 0.013019218846869188 0.08307580660388506 0.08907563025210231
merge complete for batch_idx 805 (903) patterns
start get explainer for batch_idx 805
0.0 in: 0.0
12.41 in: 1014.2196536064148
24.81 in: 888.2509303092957
37.22 in: 968.872927904129
49.63 in: 854.1463894844055
62.03 in: 969.742495059967
74.44 in: 999.1439106464386
86.85 in: 999.951663017273
99.26 in: 969.089480638504
gs 0.07643357203110702 , 3.576923076923077 , 0.9094292803970223 , 1.941011206387882e-05
ls 0.2478972840175114 , 2.3076923076923075 , 0.8071217221120079 , 2.2094189721360693
an 0.11205197987796257 , 3.3498759305210917 , 0.02913345596561175 , 6.845283479016117
lf 0.001814862476833185 , 6.794044665012407 , 1.0 , 0.3438901108488551
df 0.9863523573201016 , 13.53225806451613 , 0.7729528535980149 , 0.16990331266417397
ch 0.35068561471347065 , 2.1029776674937963 , 0.882293